In [43]:
from vertexai import generative_models
from vertexai.generative_models import GenerativeModel
import json

In [2]:
model = GenerativeModel(model_name="gemini-1.0-pro-vision")

In [15]:
property_functions = {
    "dihedral angle": "no",
    "rmsf": "yes",
}

In [29]:
def func_check_property(prop):
    """Function that checks if property is already present in list of properties"""
    if prop.lower() in property_functions.keys():
        if property_functions[prop.lower()] == "yes":
            return True
        else:
            return False
    else:
        return False

def write_function(prop):
    # Ask Gemini to write new function to calculate property
    new_func = model.generate_content(f'Please write me a Python function to calculate property {prop} such that I can pass it through pythons exec command')
    # Run function
    # exec(new_func.text)
    # Update dictionary
    property_functions[prop.lower()] = "yes"
    # Return function
    return new_func
    

In [11]:
## Take in question
response = model.generate_content('Can you calculate the dihedral angle of a protein xyz for me?')

In [40]:
follow_up_response = model.generate_content(f'which property is being asked for? which protein is it asking me to do this for? Give answer in JSON {response.text}')
# new_function = model.generate_content(f'write me a function to calculate this property: {follow_up_response.text}')

In [48]:
print(follow_up_response.text)

```python
{
  "property": "dihedral angle",
  "protein": "protein_xyz"
}
```


In [46]:
json.loads(follow_up_response.text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [30]:
property_asked_for = follow_up_response.text
if func_check_property(property_asked_for) is False:
    new_func = write_function(property_asked_for)

In [36]:
exec("""
def calculate_dihedral_angle(p1, p2, p3, p4):
    import numpy as np
    
    # Convert points to numpy arrays
    p1, p2, p3, p4 = np.array(p1), np.array(p2), np.array(p3), np.array(p4)
    
    # Vectors between the points
    b1 = p2 - p1
    b2 = p3 - p2
    b3 = p4 - p3
    
    # Normal vectors to the planes containing the three bonds
    n1 = np.cross(b1, b2)
    n2 = np.cross(b2, b3)
    
    # Normalize vectors
    n1 /= np.linalg.norm(n1)
    n2 /= np.linalg.norm(n2)
    
    # Calculate the dihedral angle between the two planes
    m = np.cross(n1, n2)
    angle = np.arctan2(np.dot(m, b2 / np.linalg.norm(m)), np.dot(n1, n2))
    return np.degrees(angle)

# Example usage:
# dihedral_angle = calculate_dihedral_angle((x1, y1, z1), (x2, y2, z2), (x3, y3, z3), (x4, y4, z4))
""")

In [39]:
follow_up_response.text

'Dihedral angle'

In [28]:
print(new_function.text)

```python
def dihedral_angle(atom1, atom2, atom3, atom4):
  """
  Calculates the dihedral angle between four atoms.

  Args:
    atom1: The first atom.
    atom2: The second atom.
    atom3: The third atom.
    atom4: The fourth atom.

  Returns:
    The dihedral angle in degrees.
  """

  # Calculate the vectors between the atoms.

  v1 = atom2.position - atom1.position
  v2 = atom3.position - atom2.position
  v3 = atom4.position - atom3.position

  # Calculate the cross products of the vectors.

  c1 = np.cross(v1, v2)
  c2 = np.cross(v2, v3)

  # Calculate the dot product of the cross products.

  dot = np.dot(c1, c2)

  # Calculate the norm of the cross products.

  norm1 = np.linalg.norm(c1)
  norm2 = np.linalg.norm(c2)

  # Calculate the dihedral angle.

  angle = np.arctan2(dot, norm1 * norm2)

  # Convert the angle to degrees.

  angle = angle * 180 / np.pi

  return angle
```


In [10]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "```\nHEADER    2jof (model 2)\nTITLE     Crystal structure of the 5-azacytidine-bound form of human DNA cytosine-5 methyltransferase\nCOMPND    DNA (5\'-D(*CP*GP*AP*GP*CP*GP*AP*CP*G)-3\')\nAUTHOR    C.G. Mueller-Dieckmann, R. Schulz, W. Saenger\n\nCRYST1   48.600  48.600  75.000  90.00  90.00  90.00 P 1\nORIGIN    -24.300  -24.300  -37.500\n\nSCALE1    0.02060  0.00000  0.00000\nSCALE2    0.00000  0.02060  0.00000\nSCALE3    0.00000  0.00000  0.01333\n\nATOM      1  C1\'  DC  1    -6.502   0.837  -7.292  1.00  0.00           C\nATOM      2  C2\'  DC  1    -5.130   0.695  -7.238  1.00  0.00           C\nATOM      3  O2\'  DC  1    -4.839   1.739  -6.973  1.00  0.00           O\nATOM      4  C3\'  DC  1    -3.785   1.697  -7.635  1.00  0.00           C\nATOM      5  C4\'  DC  1    -3.487   0.603  -8.261  1.00  0.00           C\nATOM      6  C5\'  DC  1    -4.535  -0.462  -8.295  1.00  0.00           C\nATOM      7  O5\' 